In [ ]:
import torch


In [ ]:
pip install tiktoken

In [ ]:
import tiktoken

In [ ]:
with open("/content/the-verdict.txt",  "r", encoding = "utf-8") as file:
  raw_text = file.read()
raw_text[:50]

'I HAD always thought Jack Gisburn rather a cheap g'

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class GPTDatasetV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    encoded_text = tokenizer.encode(text)

    for i in range(0, len(encoded_text) - max_length, stride):
      input_chunk = encoded_text[i : i + max_length]
      target_chunk = encoded_text[i + 1 : i + max_length + 1]

      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __getitem__(self, index):
    return self.input_ids[index], self.target_ids[index]

  def __len__(self):
    return len(self.input_ids)

In [17]:
def create_dataloder_V1(text, batch_size = 4, shuffle = True, num_workers = 0,
                        drop_last = True, max_length = 4, stride = 4):

  tokenizer = tiktoken.get_encoding("gpt2")

  gpt_dataset = GPTDatasetV1(text, tokenizer, max_length, stride)

  dataloader = DataLoader(dataset = gpt_dataset,
                          batch_size = batch_size,
                          shuffle = shuffle,
                          num_workers = num_workers,
                          drop_last = True)
  return dataloader

In [19]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
token_embedding_layer

Embedding(50257, 256)

In [20]:
dataloader = create_dataloder_V1(raw_text)

In [22]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [23]:
token_embedded = token_embedding_layer(inputs)
token_embedded.shape

torch.Size([4, 4, 256])

# Position Embeddings:

In [24]:
max_length = 4

In [25]:
position_embeddings_layer = torch.nn.Embedding(max_length, output_dim)
position_embeddings_layer.weight.shape

torch.Size([4, 256])

In [26]:
position_embeddings = position_embeddings_layer(torch.arange(max_length))
position_embeddings

tensor([[ 0.4891, -0.2587, -0.2283,  ...,  1.7749, -1.6356, -1.0557],
        [ 0.9157,  0.8983, -1.2444,  ...,  0.5072, -0.8070,  1.1140],
        [ 0.1836,  0.2363, -0.6286,  ...,  0.8433,  0.9771,  0.5554],
        [ 0.5863, -2.2419, -1.0203,  ..., -0.6030,  0.2139,  0.3666]],
       grad_fn=<EmbeddingBackward0>)

In [27]:
input_embeddings = token_embedded + position_embeddings

In [29]:
input_embeddings.shape

torch.Size([4, 4, 256])